In [1]:
import pandas as pd
import sqlite3
import seaborn as sb
import matplotlib as plt

In [2]:
# SQLite database passage
conn = sqlite3.connect('space_missions.sqlite')
cur = conn.cursor()

tablename = 'records'

df = pd.read_csv('space_missions_normalized.csv')
df.to_sql(tablename, conn, if_exists='replace', index=False)

4630

In [4]:
# 
q = '''
SELECT Company FROM records
'''
pd.read_sql_query(q,conn)

,Company
0,RVSN USSR
1,RVSN USSR
2,US Navy
3,AMBA
4,US Navy
...,...
4625,SpaceX
4626,CASC
4627,SpaceX
4628,CAS Space
